<a href="https://colab.research.google.com/github/ansfarooq7/JPMorgan-Chase-Virtual-Internship/blob/master/prototypes/L4_Project_fourth_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
!pip install transformers
!pip install wikipedia
!pip install gradio

from transformers import RobertaTokenizer, RobertaForMaskedLM, GPT2Tokenizer, GPT2LMHeadModel, pipeline
import torch
import wikipedia
import re
import random
import nltk
import gradio as gr
nltk.download('cmudict')

     |████████████████████████████████| 3.8 MB 5.0 MB/s 
     |████████████████████████████████| 6.5 MB 36.2 MB/s 
     |████████████████████████████████| 596 kB 36.6 MB/s 
     |████████████████████████████████| 895 kB 19.6 MB/s 
     |████████████████████████████████| 67 kB 2.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=d3126dddf268e02d00cddc0310199c401a2221e7b487e05045c41bbd31ddaaa5
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia
     |████████████████████████████████| 572 kB 5.2 MB/s 
     |████████████████████████████████| 87 kB 6.4 MB/s 
     |████████████████████████████████| 527 kB 48.7 MB/s 
     |████████████████████████████████| 952 kB 48.0 MB/s 
     |████████████████████████████████| 397

     |████████████████████████████████| 656 kB 5.2 MB/s 
     |████████████████████████████████| 255 kB 55.0 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 84 kB 2.4 MB/s 
     |████████████████████████████████| 2.0 MB 49.8 MB/s 
     |████████████████████████████████| 210 kB 57.4 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
     |████████████████████████████████| 10.9 MB 52.8 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 3.6 MB 41.0 MB/s 
     |████████████████████████████████| 61 kB 423 kB/s 
     |████████████████████████████████| 856 kB 56.8 MB/s 
     |████████████████████████████████| 58 kB 4.7 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4712 sha256=8b28bfc1ee45f16fbc674bd07fa46e253ad6f6997d49a5b56f14a17b90836bbe

True

In [ ]:
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base')

gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=gpt2_tokenizer.eos_token_id)

gpt2_pipeline = pipeline('text-generation', model=gpt2_model, tokenizer=gpt2_tokenizer)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

## Helper functions

In [ ]:
frequent_words = set()
        
with open("wordFrequency.txt", 'r') as f:
    line = f.readline()
    while line != '':  # The EOF char is an empty string
        frequent_words.add(line.strip())
        line = f.readline()

def filter_rhymes(word):
    filter_list = ['an', 'to', 'on', 'has', 'but', 'the', 'in', 'and', 'a', 'are', 'or', 'its', 'it''s'] 
    if word in filter_list:
        return False
    else:
        return True

def remove_punctuation(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace("\n", " ")
    return text.strip()

def get_rhymes(inp, level):
    entries = nltk.corpus.cmudict.entries()
    syllables = [(word, syl) for word, syl in entries if word == inp]
    rhymes = []
    filtered_rhymes = set()
    for (word, syllable) in syllables:
        rhymes += [word for word, pron in entries if pron[-level:] == syllable[-level:]]
    
    for word in rhymes:
        if (word in frequent_words) and (word != inp):
            filtered_rhymes.add(word)
    return filtered_rhymes

def get_inputs_length(input):
    input_ids = gpt2_tokenizer(input)['input_ids']
    return len(input_ids)

## RoBERTa

In [ ]:
def get_prediction(sent):
    
    token_ids = roberta_tokenizer.encode(sent, return_tensors='pt')
    masked_position = (token_ids.squeeze() == roberta_tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position ]

    with torch.no_grad():
        output = roberta_model(token_ids)

    last_hidden_state = output[0].squeeze()

    list_of_list =[]
    for index,mask_index in enumerate(masked_pos):
        words = []
        while not words:
            mask_hidden_state = last_hidden_state[mask_index]
            idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
            for i in idx:
                word = roberta_tokenizer.decode(i.item()).strip()
                if (remove_punctuation(word) != "") and (word != '</s>'):
                    words.append(word)
        list_of_list.append(words)
        print(f"Mask {index+1} Guesses: {words}")
    
    best_guess = ""
    for j in list_of_list:
        best_guess = best_guess+" "+j[0]
        
    return best_guess

## GPT-2 + RoBERTa

In [ ]:
def get_line(prompt, inputs_len):
    output = gpt2_pipeline(
        prompt + ".",
        min_length=4,
        max_length=inputs_len + 7,
        clean_up_tokenization_spaces=True,
        return_full_text=False
    )
    return remove_punctuation(output[0]['generated_text'])

def get_rhyming_line(prompt, rhyming_word, inputs_len):
    output = gpt2_pipeline(
        prompt + ".",
        min_length=4,
        max_length=inputs_len + 3,
        clean_up_tokenization_spaces=True,
        return_full_text=False
    )
    gpt2_sentence = remove_punctuation(output[0]['generated_text'])
    while len(gpt2_sentence) == 0:
        output = gpt2_pipeline(
            prompt + ".",
            min_length=4,
            max_length=inputs_len + 3,
            clean_up_tokenization_spaces=True,
            return_full_text=False
        )
        gpt2_sentence = remove_punctuation(output[0]['generated_text'])
    
    print(f"\nGetting rhyming line starting with '{gpt2_sentence}' and ending with rhyming word '{rhyming_word}'")
    sentence = gpt2_sentence + " ___ ___ ___ " + rhyming_word
    print(f"Original Sentence: {sentence}")
    if sentence[-1] != ".":
        sentence = sentence.replace("___","<mask>") + "."
    else:
        sentence = sentence.replace("___","<mask>")
    print(f"Original Sentence replaced with mask: {sentence}")
    print("\n")
 
    predicted_blanks = get_prediction(sentence)
    print(f"\nBest guess for fill in the blanks: {predicted_blanks}")
    final_sentence = gpt2_sentence + predicted_blanks + " " + rhyming_word
    print(f"Final Sentence: {final_sentence}")
    return final_sentence

def gpt2_summary(topic):
    output = gpt2_pipeline(
        f"Here is some information about {topic}.",
        min_length=100,
        max_length=300,
        clean_up_tokenization_spaces=True,
        return_full_text=False
    )
    return remove_punctuation(output[0]['generated_text'])

## Limerick generation

In [ ]:
def generate(topic, wiki=True):
    if wiki:
        try:
            topic_search = wikipedia.search(topic, results=3)
            print(f"Wikipedia search results for {topic} are: {topic_search}")
            topic_summary = remove_punctuation(wikipedia.summary(topic_search[0], auto_suggest=False))
        except wikipedia.DisambiguationError as e:
            print(f"Wikipedia returned a disambiguation error for {topic}. Selecting the first option {e.options[0]} instead.")
            page = e.options[0]
            topic_summary = remove_punctuation(wikipedia.summary(page, auto_suggest=False))
        except:
            return(f"Method A struggled to find information about {topic}, please try a different topic!")
    else:
        topic_summary = remove_punctuation(gpt2_summary(topic))

    word_list = topic_summary.split()
    topic_summary_len = len(topic_summary)
    no_of_words = len(word_list)
    inputs_len = get_inputs_length(topic_summary)
    print(f"Topic Summary: {topic_summary}")
    print(f"Topic Summary Length: {topic_summary_len}")
    print(f"No of Words in Summary: {no_of_words}")
    print(f"Length of Input IDs: {inputs_len}")         

    rhyming_words_125 = []
    while len(rhyming_words_125) < 3 or valid_rhyme == False or len(first_line) == 0:
        first_line = get_line(topic_summary, inputs_len)
        if first_line:
            end_word = remove_punctuation(first_line.split()[-1])
            valid_rhyme = filter_rhymes(end_word)
            if valid_rhyme:
                print(f"\nFirst Line: {first_line}")
                rhyming_words_125 = list(get_rhymes(end_word, 3))
                print(f"Rhyming words for '{end_word}' are {rhyming_words_125}")
                limerick = first_line + "\n"

    rhyming_word = random.choice(rhyming_words_125)
    rhyming_words_125.remove(rhyming_word)
    prompt = topic_summary + " " + first_line
    inputs_len = get_inputs_length(prompt)
    print(f"Prompt: {prompt}")
    print(f"Length of prompt: {inputs_len}")
    second_line = get_rhyming_line(prompt, rhyming_word, inputs_len)
    print(f"\nSecond Line: {second_line}")
    limerick += second_line + "\n"

    rhyming_words_34 = []
    prompt = prompt + " " + second_line
    inputs_len = get_inputs_length(prompt)
    print(f"Prompt: {prompt}")
    print(f"Length of prompt: {inputs_len}")
    while len(rhyming_words_34) < 2 or valid_rhyme == False or len(third_line) == 0:
        third_line = get_line(prompt, inputs_len)
        if third_line:
            print(f"\nThird Line: {third_line}")
            end_word = remove_punctuation(third_line.split()[-1])
            valid_rhyme = filter_rhymes(end_word)
            print(f"Does '{end_word}' have valid rhymes: {valid_rhyme}")
            rhyming_words_34 = list(get_rhymes(end_word, 3))
            print(f"Rhyming words for '{end_word}' are {rhyming_words_34}")
            if valid_rhyme and len(rhyming_words_34) > 1:
                limerick += third_line + "\n"

    rhyming_word = random.choice(rhyming_words_34)
    rhyming_words_34.remove(rhyming_word)
    prompt = prompt + " " + third_line
    inputs_len = get_inputs_length(prompt)
    print(f"Prompt: {prompt}")
    print(f"Length of prompt: {inputs_len}")
    fourth_line = get_rhyming_line(prompt, rhyming_word, inputs_len)
    print(f"\nFourth Line: {fourth_line}")
    limerick += fourth_line + "\n"

    rhyming_word = random.choice(rhyming_words_125)
    rhyming_words_125.remove(rhyming_word)
    prompt = prompt + " " + fourth_line
    inputs_len = get_inputs_length(prompt)
    print(f"Prompt: {prompt}")
    print(f"Length of prompt: {inputs_len}")
    fifth_line = get_rhyming_line(prompt, rhyming_word, inputs_len)
    print(f"\nFifth Line: {fifth_line}")
    limerick += fifth_line + "\n"

    print("\n")
    print(limerick)

    return limerick

In [ ]:
def compare_summaries(topic):
    wiki_limerick = generate(topic)
    gpt2_limerick = generate(topic, wiki=False)

    output1 = wiki_limerick
    output2 = gpt2_limerick
    print(output1 + "\n" + output2)

    return output1, output2

In [ ]:
description = "Generates limericks (five-line poems with a rhyme scheme of AABBA) using two different methods, please be patient as it can take up to a minute to generate both limericks."
article = '<center><big><strong>Limerick Generation</strong></big></center>'\
        '<center><strong>By Ans Farooq</strong></center>'\
        '<center><small>Level 4 Individual Project</small></center>'\
        '<center><small>BSc Computing Science</small></center>'\
        '<center><small>University of Glasgow</small></center>'\
        '<strong>Description</strong><br>'\
        'Recent advances in natural language processing (NLP) have shown '\
        'incredible promise at generating human-quality language. Poetry '\
        'presents an additional challenge as it often relies on rhyme and '\
        'rhythm of language. Factoring these in presents an interesting '\
        'challenge to new deep learning-based methods. This text-generation '\
        'project examines the use of transformer-based deep learning methods '\
        'and the addition of constraints for length, rhyme and rhythm given '\
        'example words to seed a poem. This interface allows you to produce two '\
        'cohesive limericks automatically, using two different methods. The '\
        'results of this project are to be evaluated through human comparisons.'

gr_input = gr.inputs.Textbox(label='Topic')
gr_output1 = gr.outputs.Textbox(label='Method A')
gr_output2 = gr.outputs.Textbox(label='Method B')

interface = gr.Interface(
    fn=compare_summaries, 
    inputs=gr_input, 
    outputs=[gr_output1, gr_output2],
    title="Text-generation with rhyme and rhythm",
    layout="horizontal",
    theme="peach",
    description=description,
    article=article)
interface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://55980.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Wikipedia search results for cricket are: ['Cricket', 'ESPNcricinfo', 'International Cricket Council']
Topic Summary: Cricket is a batandball game played between two teams of eleven players each on a field at the centre of which is a 22yard 20metre pitch with a wicket at each end each comprising two bails balanced on three stumps The game proceeds when a player on the fielding team called the bowler bowls propels the ball from one end of the pitch towards the wicket at the other end with an over being completed once they have legally done so six times The batting side has one player at each end of the pitch with the player at the opposite end of the pitch from the bowler aiming to strike the ball with a bat The batting side scores runs either when the ball reaches the boundary of the field or when the two batters swap ends of the pitch which results in one run The fielding sides aim is to prevent runscoring and dismiss each batter so they are out and are said to have lost their wicket 

(<fastapi.applications.FastAPI at 0x7f85756fa690>,
 'http://127.0.0.1:7860/',
 'https://55980.gradio.app')